In [3]:
import time
import pandas as pd
import json

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [4]:
BASE_URL = 'https://www.raexpert.ru'

In [6]:
def collect_data(web_element) -> None:
    soup = BeautifulSoup(web_element.get_attribute('outerHTML'), "html.parser")
    for row in soup.find_all('tr'):
        temp = dict()
        columns = row.find_all('td')
        links = row.find_all('a')
        temp['name'] = columns[0].text[:len(columns[0].text)//2].strip()
        temp['name_link'] = BASE_URL + links[0].get('href')
        temp['rating'] = columns[1].text
        temp['forecast'] = columns[2].text
        temp['date'] = columns[3].text
        temp['press_release_link'] = BASE_URL + links[1].get('href')
        global data
        data.append(temp)

In [7]:
def get_text(web_element, idx) -> None:
    soup = BeautifulSoup(web_element.get_attribute('outerHTML'), "html.parser")
    paragraphs = soup.find_all('p')
    result = ''
    for paragraph in paragraphs:
        result += (paragraph.text + '\n')
    global data
    data[idx]['press_release_text'] = result

In [5]:
data = list()

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

browser = webdriver.Chrome(options=options)
browser.get('https://www.raexpert.ru/ratings/bankcredit_all/')
time.sleep(1)

print('Starting...')

count = 1
while True:
    try:
        browser.find_element(By.XPATH, f'/html/body/main/div/div[2]/div/div[6]/span[{count}]').click()
        table = browser.find_element(By.XPATH, '/html/body/main/div/div[2]/div/div[5]/table/tbody')
        collect_data(table)
        time.sleep(1)
        if count < 7:
            count += 1
    except:
        browser.quit()
        break
    print('#', end='')

print('Parsing has finished')


In [109]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

browser = webdriver.Chrome(options=options)

for i in tqdm(range(len(data))):
    browser.get(data[i]['press_release_link'])
    time.sleep(1)
    text = browser.find_element(By.XPATH, '/html/body/main/div/div[2]/div[1]/div')
    get_text(text, i)

browser.quit()

In [111]:
data_json = json.dumps(data, indent = 4) 

In [113]:
df = pd.read_json(data_json)

c:\Users\wilte\anaconda3\lib\site-packages\pandas\io\json\_json.py:1026: UserWarning: Parsing '30.08.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  new_data = to_datetime(new_data, errors="raise", unit=date_unit)
c:\Users\wilte\anaconda3\lib\site-packages\pandas\io\json\_json.py:1026: UserWarning: Parsing '29.08.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  new_data = to_datetime(new_data, errors="raise", unit=date_unit)
c:\Users\wilte\anaconda3\lib\site-packages\pandas\io\json\_json.py:1026: UserWarning: Parsing '28.08.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  new_data = to_datetime(new_data, errors="raise", unit=date_unit)
c:\Users\wilte\anaconda3\lib\site-packages\pandas\io\json\_json.py:1026: UserWarning: Parsing '23.08.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True

In [114]:
df.sample(5)

,name,name_link,rating,forecast,date,press_release_link,press_release_text
86,"ПАО ""МТС-БАНК""",https://www.raexpert.ru/database/companies/mbrr,ruA,Стабильный,2023-01-03,https://www.raexpert.ru/releases/2023/mar01e,"Москва, 1 марта 2023 г.\nРейтинговое агентст..."
199,"АО КБ ""АССОЦИАЦИЯ""",https://www.raexpert.ru/database/companies/kb_ac,отозван,—,2019-07-29,https://www.raexpert.ru/releases/2019/jul29,"Москва, 29 июля 2019 г.\n«Эксперт РА» отозвал..."
53,"ПАО БАНК ""СИАБ""",https://www.raexpert.ru/database/companies/siab,ruB,Стабильный,2023-04-20,https://www.raexpert.ru/releases/2023/apr20e,"Москва, 20 апреля 2023 г.\nРейтинговое агентст..."
67,"АО ""МБ БАНК""",https://www.raexpert.ru/database/companies/100...,ruBBB-,Стабильный,2023-03-31,https://www.raexpert.ru/releases/2023/mar31,"Москва, 31 марта 2023 г.\nРейтинговое агентств..."
134,"АО ""БМ-БАНК""",https://www.raexpert.ru/database/companies/100...,ruA-,Стабильный,2022-10-28,https://www.raexpert.ru/releases/2022/oct28,"Москва, 28 октября 2022 г.\nРейтинговое агентс..."


In [2]:
df.to_csv('../data_output/ra_banks.csv')

NameError: name 'df' is not defined